# Lab 3: Ephemeral Inline Volumes

CSI allows the declaration of ephemeral volumes "inline" within a `Pod`. This is an extension of the idea of using the `emptyDir` volume concept for storing data temporarily on the Kubernetes worker node and outside of the layered filesystem where the container is running.

With ephemeral inline volumes, a CSI driver can be used to declare a temporary scratch disk, offloading capacity and performance from the Kubernetes worker node.

## Direct access to the CSI driver

One of the drawbacks of allowing users to declare inline volumes is that direct access to the CSI driver is needed. Inline volumes do not use `PersistentVolumeClaims` nor a `StorageClass`, hence all the details need to reside inline the declaration.

A `Secret` is needed, either in the same `Namespace` or in a foreign `Namespace` from where the inline declaration is being made. We'll use the same `Namespace` for this exercise.

There should be a `Secret` declared in your tenant `Namespace`.

In [ ]:
kubectl get secret/csi-secret -o yaml

`Secrets` store attribute values in base64 encoding and only protects from casual over-the-shoulder eavesdropping. For example, to reveal the `servicePort`, run the following.

In [ ]:
echo -n ODA4MA== | base64 -d

## Declare a workload

In this three replica `Deployment`, we're kicking off a logging process in three `Pods`. 

---
```yaml
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-inline-deploy
  labels:
    app: my-inline-deploy
spec:
  replicas: 3
  strategy:
    type: Recreate
  selector:
    matchLabels:
      app: my-logger
  template:
    metadata:
      labels:
        app: my-logger
    spec:
      containers:
      - name: my-logger
        image: busybox
        command: ["bin/sh"]
        args: ["-c", "while true; do date >> /data/mydata.txt; sleep 1; done"]
        volumeMounts:
        - name: inline
          mountPath: /data
      volumes:
      - name: inline
        csi:
          driver: csi.hpe.com
          nodePublishSecretRef:
            name: csi-secret
          fsType: xfs
          volumeAttributes:
            csi.storage.k8s.io/ephemeral: "true"
            accessProtocol: "iscsi"
            size: "32Mi"
```
---

In the `.spec.template.spec.volumes` is where the action is. The `volumeAttributes` keys are actual `StorageClass` parameters supported by the HPE CSI Driver and the CSP being used in the backend. This opens up very creative ways to temporarily access data.

Let's declare the workload.

In [ ]:
kubectl create -f obj/my-inline-deploy.yaml
kubectl rollout status deploy/my-inline-deploy

Let's wait for the `Deployment` to come up, wait to see `3/3`.

In [ ]:
kubectl get deploy/my-inline-deploy

Let's check for `PersistentVolumeClaims`.

In [ ]:
kubectl get pvc

If all objects have been cleaned up from prior exercises, there should not be any `PersistentVolumeClaims` associated with our `Deployment`.

Inspect the `Pods` in the `Deployment`.

In [ ]:
kubectl get pods -o wide -l app=my-logger

The key here is that we should have three `Pods` running on different nodes, illustrating the fact that inline volumes are all local to each of the `Pods`. Check the mountpoint inside the first `Pod`.

In [ ]:
kubectl exec -it -c my-logger deploy/my-inline-deploy -- df -h /data

Once the `Deployment` is deleted, the backend volumes will be deleted. If a `Pod` gets rescheduled or restarted, a new volume will be created and the old one deleted.

In [ ]:
kubectl delete deploy/my-inline-deploy

# Summary

Ephemeral inline volumes are useful for many things. However, there is a security concern that needs to be addressed by creating a separate tenant on the backend system to ensure that users do not get privileged access to the backend system.

Let's continue to [Lab 4: Non-portable HPE CSI Driver Capabilities](4-WKSHP-CSI-HPE.ipynb)

> **Learn more**
> - {{ BRANDING }} Community Blog: [Using Raw Block and Ephemeral Inline Volumes on Kubernetes](https://developer.hpe.com/blog/EE2QnZBXXwi4o7X0E4M0/using-raw-block-and-ephemeral-inline-volumes-on-kubernetes)